In [ ]:
from IPython.display import clear_output
from time import sleep

import numpy as np
import pandas as pd
from scipy.linalg import null_space
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import expm_multiply
import matplotlib.pyplot as plt
import networkx as nx
import pyvis.network as nt
import igraph

from qlinks.lattice.square_lattice import SquareLattice
from qlinks.symmetry.gauss_law import GaussLaw
from qlinks.solver.deep_first_search import DeepFirstSearch
from qlinks.computation_basis import ComputationBasis
from qlinks.model import QuantumLinkModel
from qlinks.visualizer.graph import GraphVisualizer

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_rows', None)

In [ ]:
def format_custom_index(index):
    return [f"({i}) {idx}" for i, idx in enumerate(index)]

def setup_dimer_model(lattice_shape, n_solution, coup_j, coup_rk):
    gauss_law = GaussLaw.from_staggered_charge_distri(*lattice_shape)
    gauss_law.flux_sector = (0, 0)
    dfs = DeepFirstSearch(gauss_law, max_steps=int(1e+8))
    basis = gauss_law.to_basis(dfs.solve(n_solution))
    model = QuantumLinkModel(coup_j, coup_rk, lattice_shape, basis)
    return basis, model

def setup_link_model(lattice_shape, n_solution, coup_j, coup_rk):
    gauss_law = GaussLaw.from_zero_charge_distri(*lattice_shape)
    gauss_law.flux_sector = (0, 0)
    dfs = DeepFirstSearch(gauss_law, max_steps=int(1e+8))
    basis = gauss_law.to_basis(dfs.solve(n_solution))
    model = QuantumLinkModel(coup_j, coup_rk, lattice_shape, basis)
    return basis, model

# QDM Lattice 4x2

In [ ]:
coup_j, coup_rk = (1, -0.7)
basis, model = setup_dimer_model(lattice_shape=(4, 2), n_solution=16, coup_j=coup_j, coup_rk=coup_rk)
evals, evecs = np.linalg.eigh(model.hamiltonian)

In [ ]:
df = basis.dataframe.copy(deep=True)
df.index = format_custom_index(df.index)
df

In [ ]:
evecs_df = pd.DataFrame.from_dict(
    {
        "eval": evals, 
        "kin": [(evec.T @ model.kinetic_term @ evec).item() for evec in evecs.T],
        "pot": [(evec.T @ model.potential_term @ evec).item() for evec in evecs.T],
    }
)
evecs_df[evecs_df["kin"].abs() < 1e-12]

In [ ]:
evecs_df.plot(style='o', grid=True)
# plt.axvline(np.pi, linestyle='--', color='gray')
# plt.axvline(2 * np.pi, linestyle='--', color='gray')
# plt.axvline(3 * np.pi, linestyle='--', color='gray')
# plt.axvline(4 * np.pi, linestyle='--', color='gray')

In [ ]:
k_rank = np.linalg.matrix_rank(model.kinetic_term)
h_rank = np.linalg.matrix_rank(model.hamiltonian)
print(k_rank, h_rank, basis.n_states, (basis.n_states - k_rank) / basis.n_states)

In [ ]:
plt.spy(evecs, precision=1e-12, markersize=12)
plt.show()

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
# color = ["orange" for i in range(ig.vcount())]
# highlight = [
#     [14, 13, 11, 7],
#     [1, 8, 2, 4],
# ]
# highlight_color = ["aqua", "yellowgreen"]
# for i, nodes in enumerate(highlight):
#     for node in nodes:
#         color[node] = highlight_color[i]
# ig.vs["color"] = color
# sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 4)[0])
# fig, ax = plt.subplots(figsize=(6, 6), facecolor="white")
igraph.plot(
    ig,
    layout=ig.layout_kamada_kawai(),
    vertex_size=16,
    vertex_color="orange",
    # target=ax
)

In [ ]:
net = nt.Network(filter_menu=True, select_menu=True)
net.from_nx(g)
net.show_buttons(filter_=['nodes', 'edges', 'physics'])
net.write_html("nx.html")

In [ ]:
two_steps_mat = np.linalg.matrix_power(model.kinetic_term, 2)
sort_idx = np.argsort(np.diag(two_steps_mat))

plt.plot(np.diag(two_steps_mat[sort_idx, :][:, sort_idx]), linestyle="--", marker="o")
plt.show()

In [ ]:
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 3)[0])
igraph.plot(
    sub_ig,
    # layout=ig.layout_kamada_kawai(),
    # vertex_size=8,
    vertex_color="orange",
    # edge_width=0.4,
    # edge_color="darkgray",
    # vertex_label_size=6,
    inline=True
)

In [ ]:
two_steps_mat = two_steps_mat[sort_idx, :][:, sort_idx]

mask = np.where(np.diag(two_steps_mat) == 3)[0]
two_steps_mat = two_steps_mat[mask[0]:mask[-1] + 1, mask[0]:mask[-1] + 1]
# two_steps_mat[two_steps_mat < 2] = 0

print(model.kinetic_term.shape[0] - np.linalg.matrix_rank(model.kinetic_term))
print(two_steps_mat.shape[0] - np.linalg.matrix_rank(two_steps_mat))

plt.matshow(two_steps_mat)
plt.colorbar()

In [ ]:
evals2, evecs2 = np.linalg.eigh(two_steps_mat)
print(evals2)

plt.matshow(evecs2)
plt.colorbar()
null_space(two_steps_mat)

In [ ]:
entropies = np.asarray([model.entropy(evecs[:, i], 1, 0) for i in range(basis.n_states)])
entropy_df = pd.Series(entropies, name="entropy")
entropy_df

In [ ]:
entropy_df.plot(style='o')

In [ ]:
i = 5
plt.plot(evecs[:, i], linestyle='--', marker='o')
np.where(np.abs(evecs[:, i]) > 0.2)[0]

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(16, 16), facecolor="white")
ax = axes.flatten()

for i in range(basis.n_states):
    g = GraphVisualizer(SquareLattice(*model.shape, basis.dataframe.iloc[i]))
    g.plot(show=False, ax=ax[i], node_size=800)
    ax[i].set_title(f"{df.index[i]}")

In [ ]:
coup_j = np.ones((np.prod(model.shape), 1))
coup_j[0] = 1.1
energy_lump_model = QuantumLinkModel(coup_j, coup_rk, model.shape, basis)
energy_lump_ham = energy_lump_model.hamiltonian

scar_state = np.zeros((basis.n_states, 1))
scar_state[np.array([2, 4, 7, 11]), 0] = np.array([1, -1, 1, -1])
scar_state /= np.linalg.norm(scar_state)

start, stop, num = 0, 1000, 100
evol_states = expm_multiply(-1j * energy_lump_ham, scar_state, start=start, stop=stop, num=num)
fidelity = [np.abs(evol_state.T @ scar_state).item() ** 2 for evol_state in evol_states]
entropy = [model.entropy(evol_state.flatten(), 1, 0) for evol_state in evol_states]

plt.plot(np.linspace(start, stop, num), fidelity, linestyle="--", marker="o")
plt.ylim(0.0001, 1.1)
# plt.yscale('log')
plt.xlabel("t")
plt.ylabel("Fidelity(t)")
plt.tight_layout()
plt.show()

plt.plot(np.linspace(start, stop, num), np.round(entropy, 12), linestyle="--", marker="o")
plt.xlabel("t")
plt.ylabel("EE(t)")
plt.tight_layout()
plt.show()

final_state = np.real_if_close(evol_states[-1], tol=1e-12)
print(f"final fidelity = {fidelity[-1]}")
print(f"O_kin @ psi_scar = {(model.kinetic_term @ final_state).flatten()}")
print(f"<O_kin> = {(final_state.T @ model.kinetic_term @ final_state).item()}")
print(f"<O_pot> = {(final_state.T @ model.potential_term @ final_state).item()}")
print(f"<H> = {(final_state.T @ model.hamiltonian @ final_state).item()}")

# QDM Lattice 4x4

In [ ]:
coup_j, coup_rk = (1, -0.7)
basis, model = setup_dimer_model(lattice_shape=(4, 4), n_solution=132, coup_j=coup_j, coup_rk=coup_rk)
evals, evecs = np.linalg.eigh(model.hamiltonian.toarray())

In [ ]:
df = basis.dataframe.copy(deep=True)
df.index = format_custom_index(df.index)
df

In [ ]:
evecs_df = pd.DataFrame.from_dict(
    {
        "eval": evals, 
        "kin": [(evec.T @ model.kinetic_term @ evec).item() for evec in evecs.T],
        "pot": [(evec.T @ model.potential_term @ evec).item() for evec in evecs.T],
    }
)
evecs_df[evecs_df["kin"].abs() < 1e-12]

In [ ]:
evecs_df.plot(style='.', grid=True)

In [ ]:
k_rank = np.linalg.matrix_rank(model.kinetic_term)
h_rank = np.linalg.matrix_rank(model.hamiltonian)
print(k_rank, h_rank, basis.n_states, (basis.n_states - k_rank) / basis.n_states)

In [ ]:
plt.spy(evecs, precision=1e-12, markersize=0.1)
plt.show()

In [ ]:
plt.matshow(evecs[:, 72:81].T)

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    # [34, 65, 86, 124],
    # [96, 108, 107, 126],
    # [93, 115, 92, 114],
    # [77, 74, 15, 5],
    # [59, 101, 58, 100],
    # [25, 67, 26, 68],
    # [55, 37, 32, 7],
    # [43, 21, 44, 22]
    # np.where(np.abs(evecs[:, 78]) > 0.05)[0],  # type-I, <pot> = 4
    # np.where(np.abs(evecs[:, 43]) > 0.15)[0],  # type-I, <pot> = 6
    # np.where(np.diag(two_steps_mat) == 4)[0],
    # np.where(np.diag(two_steps_mat) == 6)[0],
]
highlight_color = [
    "aqua",
    # "deepskyblue",
    # "yellowgreen",
    # "royalblue",
    # "blueviolet",
    # "cornflowerblue",
    # "limegreen",
    # "pink"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
# fig, ax = plt.subplots(figsize=(10, 10), facecolor="white")
# sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 4)[0])
# sub_ig = ig.induced_subgraph(np.where(np.abs(evecs[:, 43]) > 0.1)[0])
igraph.plot(
    ig,
    layout=ig.layout_kamada_kawai(),
    vertex_size=12,
    # vertex_color="orange",
    vertex_label_size=8,
    # target=ax
)

In [ ]:
net = nt.Network(filter_menu=True, select_menu=True, font_color="black")
net.from_nx(g)
for node in net.nodes:
    node["title"] = str(node["id"])
net.show_buttons(filter_=['nodes', 'edges', 'physics'])
net.write_html("nx.html")

In [ ]:
two_steps_mat = np.linalg.matrix_power(model.kinetic_term, 2)
sort_idx = np.argsort(np.diag(two_steps_mat))

plt.plot(np.diag(two_steps_mat[sort_idx, :][:, sort_idx]), linestyle="--", marker="o")
plt.show()

In [ ]:
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 4)[0])
igraph.plot(
    sub_ig,
    # layout=sub_ig.layout_kamada_kawai(),
    # vertex_size=8,
    vertex_color="orange",
    # edge_width=0.4,
    # edge_color="darkgray",
    # vertex_label_size=6,
    inline=True
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
two_steps_mat = two_steps_mat[sort_idx, :][:, sort_idx]

mask = np.where(np.diag(two_steps_mat) == 4)[0]
two_steps_mat = two_steps_mat[mask[0]:mask[-1] + 1, mask[0]:mask[-1] + 1]
# two_steps_mat = two_steps_mat - np.diagflat(np.diag(two_steps_mat))
two_steps_mat[two_steps_mat < 2] = 0

print(model.kinetic_term.shape[0] - np.linalg.matrix_rank(model.kinetic_term))
print(two_steps_mat.shape[0] - np.linalg.matrix_rank(two_steps_mat))

plt.matshow(two_steps_mat)
plt.colorbar()

In [ ]:
two_steps_mat = nx.to_numpy_array(sub_ig.subgraph(sub_components[0]).to_networkx())
# two_steps_mat = two_steps_mat - np.diagflat(np.diag(two_steps_mat))
two_steps_mat[two_steps_mat < 2] = 0
    
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
igraph.plot(
    ig,
    # layout=sub_ig.layout_kamada_kawai(),
    # vertex_size=8,
    vertex_color="orange",
    # edge_width=0.4,
    # edge_color="darkgray",
    # vertex_label_size=6,
    inline=True
)

In [ ]:
sub_components = ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
entropies = np.asarray([model.entropy(evecs[:, i], 2, 0) for i in range(basis.n_states)])
entropy_df = pd.Series(entropies, name="entropy")
entropy_df[entropy_df < 1.9]

In [ ]:
entropy_df.plot(style='o', grid=True)

In [ ]:
i = 43
plt.plot(evecs[:, i], linestyle='--', marker='o')
np.where(np.abs(evecs[:, i]) > 0.1)[0]

In [ ]:
# {i: len(np.where(np.abs(evecs[:, i]) > 1e-12)[0]) for i in range(300)}

fig, axes = plt.subplots(4, 4, figsize=(32, 24), facecolor="white")
ax = axes.flatten()

for i, val in enumerate(np.where(np.abs(evecs[:, 43]) > 0.1)[0][:16]):
    g = GraphVisualizer(SquareLattice(*model.shape, basis.dataframe.iloc[val]))
    g.plot(show=False, ax=ax[i], node_size=800)
    ax[i].set_title(f"{df.index[val]}")

In [ ]:
coup_j = np.ones((np.prod(model.shape), 1))
coup_j[3] = 1.1
energy_lump_model = QuantumLinkModel(coup_j, coup_rk, model.shape, basis)
energy_lump_ham = energy_lump_model.hamiltonian

# scar_state = np.zeros((basis.n_states, 1))
# scar_state[np.array([25, 26, 67, 68]), 0] = np.array([1, -1, -1, 1])
# scar_state /= np.linalg.norm(scar_state)
scar_state = evecs[:, 81]

start, stop, num = 0, 800, 100
evol_states = expm_multiply(-1j * energy_lump_ham, scar_state, start=start, stop=stop, num=num)
fidelity = [np.abs(evol_state.T @ scar_state).item() ** 2 for evol_state in evol_states]
entropy = [model.entropy(evol_state.flatten(), 2, 0) for evol_state in evol_states]

plt.plot(np.linspace(start, stop, num), fidelity, linestyle="--", marker="o")
plt.ylim(0.0001, 1.1)
# plt.yscale('log')
plt.xlabel("t")
plt.ylabel("Fidelity(t)")
plt.tight_layout()
plt.show()

plt.plot(np.linspace(start, stop, num), np.round(entropy, 12), linestyle="--", marker="o")
plt.xlabel("t")
plt.ylabel("EE(t)")
plt.tight_layout()
plt.show()

final_state = np.real_if_close(evol_states[-1], tol=1e-12)
print(f"final fidelity = {fidelity[-1]}")
print(f"O_kin @ psi_scar = {(model.kinetic_term @ final_state).flatten()}")
print(f"<O_kin> = {(final_state.T @ model.kinetic_term @ final_state).item()}")
print(f"<O_pot> = {(final_state.T @ model.potential_term @ final_state).item()}")
print(f"<H> = {(final_state.T @ model.hamiltonian @ final_state).item()}")

# QDM Lattice 6x2

In [ ]:
coup_j, coup_rk = (1, 1)
basis, model = setup_dimer_model(lattice_shape=(6, 2), n_solution=76, coup_j=coup_j, coup_rk=coup_rk)
evals, evecs = np.linalg.eigh(model.hamiltonian)

In [ ]:
df = basis.dataframe.copy(deep=True)
df.index = format_custom_index(df.index)
df

In [ ]:
evecs_df = pd.DataFrame.from_dict(
    {
        "eval": evals, 
        "kin": [(evec.T @ model.kinetic_term @ evec).item() for evec in evecs.T],
        "pot": [(evec.T @ model.potential_term @ evec).item() for evec in evecs.T],
    }
)
evecs_df[evecs_df["kin"].abs() < 1e-12]

3 type I
4 as type III A or Type II? 
or maybe these 2 types can be treated as the same 

In [ ]:
evecs_df.plot(style='.', grid=True)

In [ ]:
plt.matshow(evecs[:, 34:42].T)
plt.colorbar()

In [ ]:
k_rank = np.linalg.matrix_rank(model.kinetic_term)
h_rank = np.linalg.matrix_rank(model.hamiltonian)
print(k_rank, h_rank, basis.n_states, (basis.n_states - k_rank) / basis.n_states)

In [ ]:
plt.spy(evecs, precision=1e-12, markersize=1)
plt.show()

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    np.where(np.diag(two_steps_mat) == 2)[0],
    np.where(np.diag(two_steps_mat) == 3)[0],
    np.where(np.diag(two_steps_mat) == 4)[0],
    np.where(np.diag(two_steps_mat) == 5)[0],
    # np.where(evecs[:, 40] > 0.1)[0],
    # np.where(evecs[:, 40] < -0.1)[0],
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    "yellowgreen",
    "tomato",
    "royalblue",
    # "blueviolet",
    # "cornflowerblue",
    # "limegreen",
    "pink"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
# fig, ax = plt.subplots(figsize=(10, 8), facecolor="white")
# sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 4)[0])
igraph.plot(
    ig,
    layout=ig.layout_kamada_kawai(),
    vertex_size=14,
    # vertex_color="orange",
    vertex_label_size=10,
    # target=ax
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
np.linalg.eigh(nx.to_numpy_array(sub_ig.subgraph(sub_components[0]).to_networkx()))[1][:, 5:7]

In [ ]:
net = nt.Network(filter_menu=True, select_menu=True, font_color="black")
net.from_nx(g)
for node in net.nodes:
    node["title"] = str(node["id"])
net.show_buttons(filter_=['nodes', 'edges', 'physics'])
net.write_html("nx.html")

In [ ]:
two_steps_mat = np.linalg.matrix_power(model.kinetic_term, 2)
sort_idx = np.argsort(np.diag(two_steps_mat))

plt.plot(np.diag(two_steps_mat[sort_idx, :][:, sort_idx]), linestyle="--", marker="o")
plt.show()

In [ ]:
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    # np.where(np.diag(two_steps_mat) == 3)[0],
    # np.where(np.diag(two_steps_mat) == 4)[0],
    # np.where(np.diag(two_steps_mat) == 5)[0],
    # np.where(np.abs(evecs[:, 39]) > 0.15)[0],
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    # "tomato",
    # "yellowgreen",
    # "royalblue",
    # "blueviolet",
    # "cornflowerblue",
    # "limegreen",
    # "pink"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
sub_ig = ig.induced_subgraph(np.where((np.diag(two_steps_mat) == 5) | (np.diag(two_steps_mat) == 4))[0])
igraph.plot(
    sub_ig,
    # layout=ig.layout_kamada_kawai(),
    # vertex_size=8,
    # vertex_color="orange",
    # edge_width=0.4,
    # edge_color="darkgray",
    # vertex_label_size=6,
    inline=True
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
nx.to_numpy_array(sub_ig.subgraph(sub_components[1]).to_networkx())

In [ ]:
np.linalg.eigh(nx.to_numpy_array(sub_ig.subgraph(sub_components[0]).to_networkx()))

In [ ]:
# g = nx.from_numpy_array(two_steps_mat)
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    # list(map(int, sub_ig.subgraph(sub_components[0]).vs["label"])),
    # list(map(int, sub_ig.subgraph(sub_components[1]).vs["label"])),
    # list(map(int, sub_ig.subgraph(sub_components[2]).vs["label"])),
    np.where(np.abs(evecs[:, 36]) > 0.08)[0],
    # np.where(np.diag(two_steps_mat) == 4)[0]
    # [7, 12, 34, 41, 57, 68]
]
highlight_color = [
    # "tomato",
    "yellowgreen",
    # "royalblue",
    # "aqua",
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
# sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 4)[0])
# sub_ig = ig.induced_subgraph(np.where(np.abs(evecs[:, 40]) > 0.05)[0])
igraph.plot(
    ig,
    layout=ig.layout_kamada_kawai(),
    vertex_size=14,
    # vertex_color="orange",
    edge_width=0.2,
    # edge_color="gray"
    vertex_label_size=8,
)

In [ ]:
two_steps_mat = two_steps_mat[sort_idx, :][:, sort_idx]

mask = np.where((np.diag(two_steps_mat) == 4))[0]
two_steps_mat = two_steps_mat[mask[0]:mask[-1] + 1, mask[0]:mask[-1] + 1]
# two_steps_mat = two_steps_mat - np.diagflat(np.diag(two_steps_mat))
# two_steps_mat[two_steps_mat < 2] = 0

print(model.kinetic_term.shape[0] - np.linalg.matrix_rank(model.kinetic_term))
print(two_steps_mat.shape[0] - np.linalg.matrix_rank(two_steps_mat))

plt.matshow(two_steps_mat)
plt.colorbar()

In [ ]:
fig, axes = plt.subplots(19, 4, figsize=(30, 60), facecolor="white")
ax = axes.flatten()

for i in range(basis.n_states):
    g = GraphVisualizer(SquareLattice(*model.shape, basis.dataframe.iloc[i]))
    g.plot(show=False, ax=ax[i], node_size=800)
    ax[i].set_title(f"{df.index[i]}")

In [ ]:
entropies = np.asarray([model.entropy(evecs[:, i], 0, 0) for i in range(basis.n_states)])
entropy_df = pd.Series(entropies, name="entropy")
entropy_df.loc[(entropy_df.index >= 20) & (entropy_df.index <= 50) & (entropy_df < 1.75)]

In [ ]:
entropy_df.plot(style='o')

In [ ]:
i = 37
plt.plot(evecs[:, i], linestyle='--', marker='o')
plt.axvline(25, linestyle='--', color='gray')
plt.axvline(50, linestyle='--', color='gray')
plt.grid()
np.where(np.abs(evecs[:, i]) > 0.2)[0]

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
degree_sequence = sorted((d for n, d in g.degree(np.where(np.abs(evecs[:, 38]) > 0.01)[0])), reverse=True)
plt.plot(degree_sequence, linestyle='--', marker='o')
plt.grid()
np.mean(degree_sequence)

In [ ]:
cyc_g = nx.cycle_graph(6)
mat = 4 * np.eye(6) + 2 * nx.to_numpy_array(cyc_g)
evals2, evecs2 = np.linalg.eigh(mat)
evals2

In [ ]:
evecs2[:, 0]

In [ ]:
coup_j = np.ones((np.prod(model.shape), 1))
# coup_j[0] = 1.05
energy_lump_model = QuantumLinkModel(coup_j, coup_rk, model.shape, basis)
energy_lump_ham = energy_lump_model.hamiltonian

scar_state = np.zeros((basis.n_states, 1))
# scar_state[np.array([30, 45, 49, 26, 29, 44, 15, 52, 60, 17, 23, 62]), 0] = np.array([-1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1])  # based on 44
# scar_state[np.array([3, 11, 16, 32, 36, 47, 58, 69]), 0] = np.array([1, -1, -1, 1, 1, -1, 1, -1])  # based on 24
scar_state[np.array([6, 9, 22, 28, 39, 43]), 0] = evecs2[:, 0].flatten()
scar_state /= np.linalg.norm(scar_state)
# scar_state = evecs[:, 44]

start, stop, num = 0, 2000, 100
evol_states = expm_multiply(-1j * energy_lump_ham, scar_state, start=start, stop=stop, num=num)
fidelity = [np.abs(evol_state.T @ scar_state).item() ** 2 for evol_state in evol_states]
entropy = [model.entropy(evol_state.flatten(), 1, 0) for evol_state in evol_states]

plt.plot(np.linspace(start, stop, num), fidelity, linestyle="--", marker="o")
plt.ylim(0.0001, 1.1)
# plt.yscale('log')
plt.xlabel("t")
plt.ylabel("Fidelity(t)")
plt.tight_layout()
plt.show()

plt.plot(np.linspace(start, stop, num), entropy, linestyle="--", marker="o")
plt.xlabel("t")
plt.ylabel("EE(t)")
plt.tight_layout()
plt.show()

final_state = np.real_if_close(evol_states[0], tol=1e-12)
print(f"final fidelity = {fidelity[0]}")
print(f"O_kin @ psi_scar = {(model.kinetic_term @ final_state).flatten()}")
print(f"<O_kin> = {(final_state.T @ model.kinetic_term @ final_state).item()}")
print(f"<O_pot> = {(final_state.T @ model.potential_term @ final_state).item()}")
print(f"<H> = {(final_state.T @ model.hamiltonian @ final_state).item()}")


# QDM Lattice 6x4

In [ ]:
coup_j, coup_rk = (1, -0.7)
basis, model = setup_dimer_model(lattice_shape=(6, 4), n_solution=1456, coup_j=coup_j, coup_rk=coup_rk)
evals, evecs = np.linalg.eigh(model.hamiltonian.toarray())

In [ ]:
df = basis.dataframe.copy(deep=True)
df.index = format_custom_index(df.index)
df

In [ ]:
evecs_df = pd.DataFrame.from_dict(
    {
        "eval": evals, 
        "kin": [(evec.T @ model.kinetic_term @ evec).item() for evec in evecs.T],
        "pot": [(evec.T @ model.potential_term @ evec).item() for evec in evecs.T],
    }
)
evecs_df[evecs_df["kin"].abs() < 1e-12]

In [ ]:
evecs_df.plot(style='.', grid=True, markersize=1)

In [ ]:
k_rank = np.linalg.matrix_rank(model.kinetic_term)
h_rank = np.linalg.matrix_rank(model.hamiltonian)
print(k_rank, h_rank, basis.n_states, (basis.n_states - k_rank) / basis.n_states)

In [ ]:
plt.spy(evecs, precision=1e-12, markersize=0.01)
plt.show()

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["orange" for i in range(ig.vcount())]
highlight = [
    # [1099, 1102, 1239, 1256],
    # [604, 601, 331, 311],
    # [830, 891, 205, 117],
    # [355, 868, 808, 352],
    # [390, 367, 655, 695], 
    # [1047, 1212, 1226, 1082],
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    # "yellowgreen",
    # "royalblue",
    # "blueviolet",
    # "cornflowerblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
# sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 4)[0])
fig, ax = plt.subplots(figsize=(10, 10), facecolor="white")
igraph.plot(
    ig,
    layout=ig.layout_kamada_kawai(),
    vertex_size=32,
    # vertex_color="orange",
    edge_width=0.4,
    # edge_color="darkgray",
    vertex_label_size=6,
    target=ax
)
fig.tight_layout()
fig.savefig("qdm_6x4_kinetic_graph.png", dpi=300)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
np.linalg.eigh(nx.to_numpy_array(sub_ig.subgraph(sub_components[0]).to_networkx()))[1][:, 3:6]

In [ ]:
net = nt.Network(filter_menu=True, select_menu=True, font_color="black")
net.from_nx(g)
for node in net.nodes:
    node["title"] = str(node["id"])
net.show_buttons(filter_=['nodes', 'edges', 'physics'])
net.write_html("nx.html")

In [ ]:
two_steps_mat = np.linalg.matrix_power(model.kinetic_term, 2)
sort_idx = np.argsort(np.diag(two_steps_mat))

plt.plot(np.diag(two_steps_mat[sort_idx, :][:, sort_idx]), linestyle="--", marker="o")
plt.show()

In [ ]:
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 4)[0])
igraph.plot(
    sub_ig,
    # layout=sub_ig.layout_kamada_kawai(),
    # vertex_size=8,
    vertex_color="orange",
    # edge_width=0.4,
    # edge_color="darkgray",
    # vertex_label_size=6,
    inline=True
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
entropies = np.asarray([model.entropy(evecs[:, i], 1, 0) for i in range(basis.n_states)])
entropy_df = pd.Series(entropies, name="entropy")
entropy_df[entropy_df < 2.0]

In [ ]:
entropy_df.plot(style='o')

In [ ]:
i = 231
plt.plot(evecs[:, i], linestyle='--', marker='o')
np.where(np.abs(evecs[:, i]) > 0.075)[0]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12), facecolor="white")
ax = axes.flatten()

for i, val in enumerate(np.where(np.abs(evecs[:, 231]) > 0.075)[0][:4]):  
    #([891, 829, 1213, 354]):  #[1099, 1102, 1239, 1256]
    g = GraphVisualizer(SquareLattice(*model.shape, basis.dataframe.iloc[val]))
    g.plot(show=False, ax=ax[i], node_size=800)
    ax[i].set_title(f"{df.index[val]}")

In [ ]:
g = GraphVisualizer(SquareLattice(*model.shape, basis.dataframe.iloc[1213]))
g.plot(node_size=800)

In [ ]:
coup_j = np.ones((np.prod(model.shape), 1))
# coup_j[0] = 1.1
energy_lump_model = QuantumLinkModel(coup_j, coup_rk, model.shape, basis)
energy_lump_ham = energy_lump_model.hamiltonian

scar_state = np.zeros((basis.n_states, 1))
# scar_state[np.array([1099, 1102, 1239, 1256]), 0] = np.array([1, -1, -1, 1])
scar_state[np.array([117, 118, 205, 828, 829, 830, 831, 885, 891]), 0] = np.array([0.55255026,  0.21474467, -0.20558535, -0.21474467, -0.34696492,
        -0.20558535,  0.21474467, -0.21474467,  0.55255026])
scar_state /= np.linalg.norm(scar_state)

start, stop, num = 0, 200, 100
evol_states = expm_multiply(-1j * energy_lump_ham, scar_state, start=start, stop=stop, num=num)
fidelity = [np.abs(evol_state.T @ scar_state).item() ** 2 for evol_state in evol_states]
entropy = [model.entropy(evol_state.flatten(), 1, 0) for evol_state in evol_states]

plt.plot(np.linspace(start, stop, num), fidelity, linestyle="--", marker="o")
plt.ylim(0.0001, 1.1)
# plt.yscale('log')
plt.xlabel("t")
plt.ylabel("Fidelity(t)")
plt.tight_layout()
plt.show()

plt.plot(np.linspace(start, stop, num), entropy, linestyle="--", marker="o")
plt.xlabel("t")
plt.ylabel("EE(t)")
plt.tight_layout()
plt.show()

final_state = np.real_if_close(evol_states[0], tol=1e-12)
print(f"final fidelity = {fidelity[0]}")
print(f"O_kin @ psi_scar = {(model.kinetic_term @ final_state).flatten()}")
print(f"<O_kin> = {(final_state.T @ model.kinetic_term @ final_state).item()}")
print(f"<O_pot> = {(final_state.T @ model.potential_term @ final_state).item()}")
print(f"<H> = {(final_state.T @ model.hamiltonian @ final_state).item()}")

# QDM Lattice 8x4

In [ ]:
coup_j, coup_rk = (1, 1)  # dfs 2 mins 50 secs
basis, model = setup_dimer_model(lattice_shape=(8, 4), n_solution=17412, coup_j=coup_j, coup_rk=coup_rk) 

In [ ]:
# basis.dataframe.to_parquet("qdm_8x4_lattice.parquet", index=False)
basis = ComputationBasis.from_parquet("data/qdm_8x4_lattice.parquet")
coup_j, coup_rk = (1, 0)
model = QuantumLinkModel(coup_j, coup_rk, (8, 4), basis)

In [ ]:
df = basis.dataframe.copy(deep=True)
df.index = format_custom_index(df.index)
df

In [ ]:
evals, evecs = np.load("data/qdm_8x4_coup_j_1_coup_rk_0_eigs.npz").values()

In [ ]:
evecs_df = pd.read_parquet("data/qdm_8x4_coup_j_1_coup_rk_0_eigs.parquet")
evecs_df[(evecs_df["kin"].abs() - np.sqrt(2)).abs() < 1e-12]

In [ ]:
evecs_df[evecs_df["pot"] < 7]

In [ ]:
evecs_df.plot(style='.', grid=True, markersize=1)

In [ ]:
plt.spy(evecs, precision=1e-12, markersize=0.05)
plt.show()

In [ ]:
i = 5590
plt.plot(evecs[:, i], linestyle='--', marker='.')
np.where(np.abs(evecs[:, i]) > 0.1)[0]

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
degree_sequence = sorted((d for n, d in g.degree(np.where(np.abs(evecs[:, 5602]) > 0.0001)[0])), reverse=True)
plt.plot(degree_sequence, linestyle='--', marker='o')
plt.grid()
np.mean(degree_sequence)

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(18, 24), facecolor="white")
ax = axes.flatten()

for i, val in enumerate(np.where(np.abs(evecs[:, 5587]) > 0.01)[0][:8]):
    g = GraphVisualizer(SquareLattice(*model.shape, basis.dataframe.iloc[val]))
    g.plot(show=False, ax=ax[i], node_size=800)
    ax[i].set_title(f"{df.index[val]}")

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
igraph.plot(
    ig,
    layout=ig.layout_kamada_kawai(),
    vertex_size=4,
    vertex_color="orange",
    edge_width=0.1,
    # edge_color="gray"
)

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 7)[0])
igraph.plot(
    sub_ig,
    # layout=ig.layout_kamada_kawai(),
    vertex_size=4,
    vertex_color="orange",
    edge_width=0.1,
    # edge_color="gray"
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

nullity = []
for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    nullity.append(mat.shape[0] - np.linalg.matrix_rank(mat))
    
np.unique(nullity, return_counts=True)

In [ ]:
two_steps_mat = csr_matrix(model.kinetic_term) ** 2
# sort_idx = np.argsort(np.diag(two_steps_mat))
# 
# plt.plot(np.diag(two_steps_mat[sort_idx, :][:, sort_idx]), linestyle="--", marker="o")
# plt.show()
print(np.unique(two_steps_mat.diagonal()))
two_steps_mat = two_steps_mat.toarray()

In [ ]:
# two_steps_mat[two_steps_mat < 2] = 0
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    # np.where(np.abs(evecs[:, 5590]) > 0.1)[0],
    # np.where((np.diag(two_steps_mat) == 6))[0],
    # np.where((np.diag(two_steps_mat) == 7))[0],
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    # "yellowgreen",
    # "tomato",
    # "royalblue",
    # "cornflowerblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 8)[0])
# sub_ig = ig.induced_subgraph(np.where((np.diag(two_steps_mat) == 6) | (np.diag(two_steps_mat) == 7))[0])
igraph.plot(
    sub_ig,
    # layout=sub_ig.layout_kamada_kawai(),
    vertex_size=6,
    # vertex_color="orange",
    edge_width=0.2,
    # edge_color="darkgray",
    vertex_label_size=2,
    inline=True
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    np.where(np.diag(two_steps_mat) == 6)[0],
    np.where(np.diag(two_steps_mat) == 7)[0],
    # np.where(np.abs(evecs[:, 5587]) > 0.05)[0]
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    # "yellowgreen",
    "tomato",
    "royalblue",
    # "cornflowerblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
sub_ig = ig.induced_subgraph(np.where((np.diag(two_steps_mat) == 6) | (np.diag(two_steps_mat) == 7))[0])
igraph.plot(
    sub_ig,
    # layout=ig.layout_kamada_kawai(),
    vertex_size=6,
    # vertex_color="orange",
    edge_width=0.1,
    # edge_color="gray"
)

In [ ]:
sub_components = sub_ig.components(mode="weak")
np.unique([len(c) for c in sub_components], return_counts=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), facecolor="white")
for i, c in enumerate(sub_components): 
    sub_sub_ig = sub_ig.induced_subgraph(c)
    colors, counts = np.unique(sub_sub_ig.vs["color"], return_counts=True)
    if len(counts) == 2:
        if counts[0] == counts[1]:
            print(i)
            igraph.plot(
                sub_sub_ig,
                # layout=sub_sub_ig.layout_kamada_kawai(),
                target=ax,
                vertex_size=6,
                # vertex_color="whitesmoke",
                edge_width=0.1,
                # edge_color="gray",
                inline=True
            )
            break
plt.show()

In [ ]:
sub_gmat = np.asarray(sub_ig.induced_subgraph(sub_components[2]).get_adjacency().data)
np.linalg.eigh(sub_gmat)[0]

In [ ]:
n = 72
2 * np.cos(2 * np.arange(1, n + 1) * np.pi / n)

# Check effective matrix

In [ ]:
l = 0.0
dj = 1.1

mat = np.array(
    [
        [l, dj, dj, 0, 0, 0, 0, 0, 0],
        [dj, l, 0, 1, 1, 0, 0, 0, 0], 
        [dj, 0, l, 0, 1, 1, 0, 0, 0], 
        [0, 1, 0, l, 0, 0, 1, 0, 0], 
        [0, 1, 1, 0, l, 0, 1, 1, 0], 
        [0, 0, 1, 0, 0, l, 0, 1, 0], 
        [0, 0, 0, 1, 1, 0, l, 0, 1],
        [0, 0, 0, 0, 1, 1, 0, l, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, l]
    ]
)

np.linalg.matrix_power(mat, 2) @ np.array([[0, 1, -1, 0, 0, 0, -1, 1, 0]]).T

In [ ]:
for i in range(20):
    print(f"matrix power {i+1}")
    walker = np.linalg.matrix_power(mat, i+1)
    walker /= np.linalg.norm(walker)
    plt.matshow(walker, animated=True)
    plt.colorbar()
    plt.show()
    sleep(0.3)
    clear_output(wait=True)

# QLM Lattice 4x2

In [ ]:
coup_j, coup_rk = (1, -0.7)
basis, model = setup_link_model(lattice_shape=(4, 2), n_solution=38, coup_j=coup_j, coup_rk=coup_rk)
evals, evecs = np.linalg.eigh(model.hamiltonian)

In [ ]:
df = basis.dataframe.copy(deep=True)
df.index = format_custom_index(df.index)
df

In [ ]:
evecs_df = pd.DataFrame.from_dict(
    {
        "eval": evals, 
        "kin": [(evec.T @ model.kinetic_term @ evec).item() for evec in evecs.T],
        "pot": [(evec.T @ model.potential_term @ evec).item() for evec in evecs.T],
    }
)
evecs_df[evecs_df["kin"].abs() < 1e-12]

In [ ]:
evecs_df.plot(style='.', grid=True)

In [ ]:
k_rank = np.linalg.matrix_rank(model.kinetic_term)
h_rank = np.linalg.matrix_rank(model.hamiltonian)
print(k_rank, h_rank, basis.n_states, (basis.n_states - k_rank) / basis.n_states)

In [ ]:
plt.spy(evecs, precision=1e-12, markersize=5.0)
plt.show()

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["orange" for i in range(ig.vcount())]
highlight = [
    # [1, 13, 24, 36],
    # [5, 13, 24, 32],
    # [7, 10, 27, 30],
    # [10, 16, 21, 27],
    # [4, 8, 12, 14, 17, 18, 23, 25, 28, 34]
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    # "yellowgreen",
    # "royalblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
# sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 5)[0])
igraph.plot(
    ig,
    layout=ig.layout_kamada_kawai(),
    vertex_size=16,
    # vertex_color="orange",
)

In [ ]:
two_steps_mat = np.linalg.matrix_power(model.kinetic_term, 2)
sort_idx = np.argsort(np.diag(two_steps_mat))

plt.plot(np.diag(two_steps_mat[sort_idx, :][:, sort_idx]), linestyle="--", marker="o")
plt.show()

In [ ]:
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 5)[0])
igraph.plot(
    sub_ig,
    # layout=sub_ig.layout_kamada_kawai(),
    # vertex_size=8,
    vertex_color="orange",
    # edge_width=0.4,
    # edge_color="darkgray",
    # vertex_label_size=6,
    inline=True
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
entropies = np.asarray([model.entropy(evecs[:, i], 0, 1) for i in range(basis.n_states)])
entropy_df = pd.Series(entropies, name="entropy")
entropy_df[entropy_df < 2.5]

In [ ]:
entropy_df.plot(style='o')

In [ ]:
i = 21
plt.plot(evecs[:, i], linestyle='--', marker='o')
np.where(np.abs(evecs[:, i]) > 0.1)[0]

In [ ]:
fig, axes = plt.subplots(10, 4, figsize=(18, 40), facecolor="white")
ax = axes.flatten()

for i in range(basis.n_states):
    g = GraphVisualizer(SquareLattice(*model.shape, basis.dataframe.iloc[i]))
    g.plot(show=False, ax=ax[i], node_size=800)
    ax[i].set_title(f"{df.index[i]}")

In [ ]:
coup_j = np.ones((np.prod(model.shape), 1))
coup_j[0] = 1.1
energy_lump_model = QuantumLinkModel(coup_j, coup_rk, model.shape, basis)
energy_lump_ham = energy_lump_model.hamiltonian

scar_state = np.zeros((basis.n_states, 1))
scar_state[np.array([10, 16, 21, 27]), 0] = np.array([-1, 1, 1, -1])
# scar_state[np.array([8, 14, 17, 23, 28]), 0] = np.array([-1, 1, -1, 2, -1])
scar_state /= np.linalg.norm(scar_state)

start, stop, num = 0, 1000, 100
evol_states = expm_multiply(-1j * energy_lump_ham, scar_state, start=start, stop=stop, num=num)
fidelity = [np.abs(evol_state.T @ scar_state).item() ** 2 for evol_state in evol_states]
entropy = [model.entropy(evol_state.flatten(), 1, 0) for evol_state in evol_states]

plt.plot(np.linspace(start, stop, num), fidelity, linestyle="--", marker="o")
plt.ylim(0.0001, 1.1)
# plt.yscale('log')
plt.xlabel("t")
plt.ylabel("Fidelity(t)")
plt.tight_layout()
plt.show()

plt.plot(np.linspace(start, stop, num), np.round(entropy, 12), linestyle="--", marker="o")
plt.xlabel("t")
plt.ylabel("EE(t)")
plt.tight_layout()
plt.show()

final_state = np.real_if_close(evol_states[-1], tol=1e-12)
print(f"final fidelity = {fidelity[-1]}")
print(f"O_kin @ psi_scar = {(model.kinetic_term @ final_state).flatten()}")
print(f"<O_kin> = {(final_state.T @ model.kinetic_term @ final_state).item()}")
print(f"<O_pot> = {(final_state.T @ model.potential_term @ final_state).item()}")
print(f"<H> = {(final_state.T @ model.hamiltonian @ final_state).item()}")

# QLM Lattice 4x4

In [ ]:
coup_j, coup_rk = (1, -0.7)
basis, model = setup_link_model(lattice_shape=(4, 4), n_solution=990, coup_j=coup_j, coup_rk=coup_rk)
evals, evecs = np.linalg.eigh(model.hamiltonian.toarray())

In [ ]:
df = basis.dataframe.copy(deep=True)
df.index = format_custom_index(df.index)
df

In [ ]:
evecs_df = pd.DataFrame.from_dict(
    {
        "eval": evals, 
        "kin": [(evec.T @ model.kinetic_term @ evec).item() for evec in evecs.T],
        "pot": [(evec.T @ model.potential_term @ evec).item() for evec in evecs.T],
    }
)
evecs_df[evecs_df["kin"].abs() < 1e-12]

In [ ]:
evecs_df[(evecs_df["kin"].abs() - 2).abs() < 1e-12]

In [ ]:
evecs_df.plot(style='.', grid=True, markersize=2)

In [ ]:
k_rank = np.linalg.matrix_rank(model.kinetic_term)
h_rank = np.linalg.matrix_rank(model.hamiltonian)
print(k_rank, h_rank, basis.n_states, (basis.n_states - k_rank) / basis.n_states)

In [ ]:
plt.spy(evecs, precision=1e-12, markersize=0.05)
plt.show()

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    np.where(np.diag(two_steps_mat) == 8)[0],
    # np.where(np.diag(two_steps_mat) == 6)[0],
    # [29, 41, 89, 110, 31, 91, 140, 100]
]
highlight_color = [
    "aqua",
    # "deepskyblue",
    # "yellowgreen",
    # "tomato",
    # "royalblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
sub_ig = ig.induced_subgraph(np.where((np.diag(two_steps_mat) == 8) | (np.diag(two_steps_mat) == 9) | (np.diag(two_steps_mat) == 10))[0])
igraph.plot(
    sub_ig,
    layout=sub_ig.layout_kamada_kawai(),
    vertex_size=10,
    # vertex_color="orange",
    edge_width=0.6,
    # edge_color="gray"
    vertex_label_size=4,
)

In [ ]:
nx.write_gexf(sub_ig.to_networkx(), "data.gexf")

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
net = nt.Network(filter_menu=True, select_menu=True, font_color="black")
net.from_nx(g)
for node in net.nodes:
    node["title"] = str(node["id"])
net.show_buttons(filter_=['nodes', 'edges', 'physics'])
net.write_html("nx.html")

In [ ]:
two_steps_mat = np.linalg.matrix_power(model.kinetic_term.toarray(), 2)
sort_idx = np.argsort(np.diag(two_steps_mat))

plt.plot(np.diag(two_steps_mat[sort_idx, :][:, sort_idx]), linestyle="--", marker="o")
plt.show()

In [ ]:
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    # [35,  40,  41,  98, 110, 129, 137, 178, 183, 215, 218, 256, 257,
    #    264, 299, 303, 307, 312, 335, 338, 343, 365, 374, 419, 420, 421,
    #    454, 455, 459, 460, 470, 473, 516, 519, 529, 530, 534, 535, 568,
    #    569, 570, 615, 624, 646, 651, 654, 677, 682, 686, 690, 725, 732,
    #    733, 771, 774, 806, 811, 852, 860, 879, 891, 948, 949, 954],
    # np.where((np.abs(evecs[:, 680]) > 0.04) & (np.abs(evecs[:, 680]) < 0.08))[0],
    # np.where((np.abs(evecs[:, 680]) > 0.08) & (np.abs(evecs[:, 680]) < 0.12))[0],
    # np.where(evecs[:, 680] > 0.12)[0],
    # np.where(evecs[:, 680] < -0.12)[0],
    # np.where(np.abs(evecs[:, 678]) > 0.05)[0],
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    # "yellowgreen",
    # "tomato",
    # "royalblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 8)[0])
igraph.plot(
    sub_ig,
    # layout=sub_ig.layout_kamada_kawai(),
    vertex_size=12,
    # vertex_color="orange",
    edge_width=0.4,
    # edge_color="darkgray",
    vertex_label_size=6,
    inline=True
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
# g = nx.from_numpy_array(two_steps_mat)
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    list(map(int, sub_ig.subgraph(sub_components[0]).vs["label"])),
    list(map(int, sub_ig.subgraph(sub_components[1]).vs["label"])),
    list(map(int, sub_ig.subgraph(sub_components[2]).vs["label"])),
    # np.where(np.diag(two_steps_mat) == 8)[0]
]
highlight_color = [
    "yellowgreen",
    "tomato",
    "royalblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
# sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 8)[0])
sub_ig = ig.induced_subgraph(np.where(np.abs(evecs[:, 226]) > 0.05)[0])
igraph.plot(
    sub_ig,
    # layout=ig.layout_kamada_kawai(),
    vertex_size=10,
    # vertex_color="orange",
    edge_width=0.2,
    # edge_color="gray"
    vertex_label_size=4,
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))

In [ ]:
sub_sub_ig = sub_ig.induced_subgraph(sub_components[2])
igraph.plot(
    sub_sub_ig,
    layout=sub_sub_ig.layout_kamada_kawai(),
    vertex_size=10,
    # vertex_color="orange",
    edge_width=0.2,
    # edge_color="gray"
    vertex_label_size=4,
)

In [ ]:
net = nt.Network(filter_menu=True, select_menu=True, font_color="black")
net.from_nx(sub_ig.subgraph(sub_components[2]).to_networkx())
for node in net.nodes:
    node["title"] = str(node["id"])
net.show_buttons(filter_=['nodes', 'edges', 'physics'])
net.write_html("nx.html")

In [ ]:
nx.write_gexf(sub_ig.subgraph(sub_components[2]).to_networkx(), "data.gexf")

In [ ]:
np.linalg.eigh(nx.to_numpy_array(sub_ig.subgraph(sub_components[2]).to_networkx()))[0]

In [ ]:
np.linalg.eigh(nx.to_numpy_array(sub_ig.subgraph(sub_components[2]).to_networkx()))[1][:, 22:25]

In [ ]:
entropies = np.asarray([model.entropy(evecs[:, i], 1, 0) for i in range(basis.n_states)])
entropy_df = pd.Series(entropies, name="entropy")
entropy_df[entropy_df < 3.5]

In [ ]:
entropy_df.plot(style='o')

In [ ]:
i = 678
plt.plot(evecs[:, i], linestyle='--', marker='o')
np.where(np.abs(evecs[:, i]) > 0.05)[0]
# np.sign(evecs[:, i][idx])

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(18, 24), facecolor="white")
ax = axes.flatten()

# for i, val in enumerate(np.where(np.abs(evecs[:, 246]) > 0.1)[0][-16:]):
for i, val in enumerate(np.where(np.abs(evecs[:, 225]) > 0.05)[0][:16]):
# for i, val in enumerate(np.where(np.abs(evecs[:, 678]) > 0.1)[0][:8]):
# for i, val in enumerate([470, 954, 474, 949]):
    g = GraphVisualizer(SquareLattice(*model.shape, basis.dataframe.iloc[val]))
    g.plot(show=False, ax=ax[i], node_size=800)
    ax[i].set_title(f"{df.index[val]}")

In [ ]:
coup_j = np.ones((np.prod(model.shape), 1))
# coup_j[0] = 1.05
energy_lump_model = QuantumLinkModel(coup_j, coup_rk, model.shape, basis)
energy_lump_ham = energy_lump_model.hamiltonian

scar_state = np.zeros((basis.n_states, 1))
# scar_state[np.array([82, 216, 226, 257, 299, 333, 656, 690, 732, 763, 773, 907]), 0] = np.array([1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.])
scar_state[np.array([470, 954, 474, 949]), 0] = np.array([1, 1, 1, 1])
scar_state /= np.linalg.norm(scar_state)
# scar_state = evecs[:, 678]

start, stop, num = 0, 200, 100
evol_states = expm_multiply(-1j * energy_lump_ham, scar_state, start=start, stop=stop, num=num)
fidelity = [np.abs(evol_state.T @ scar_state).item() ** 2 for evol_state in evol_states]
entropy = [model.entropy(evol_state.flatten(), 2, 0) for evol_state in evol_states]

plt.plot(np.linspace(start, stop, num), fidelity, linestyle="--", marker="o")
plt.ylim(0.0001, 1.1)
# plt.yscale('log')
plt.xlabel("t")
plt.ylabel("Fidelity(t)")
plt.tight_layout()
plt.show()

plt.plot(np.linspace(start, stop, num), np.round(entropy, 12), linestyle="--", marker="o")
plt.xlabel("t")
plt.ylabel("EE(t)")
plt.tight_layout()
plt.show()

final_state = np.real_if_close(evol_states[0], tol=1e-12)
print(f"final fidelity = {fidelity[0]}")
print(f"O_kin @ psi_scar = {(model.kinetic_term @ final_state).flatten()}")
print(f"<O_kin> = {(final_state.T @ model.kinetic_term @ final_state).item()}")
print(f"<O_pot> = {(final_state.T @ model.potential_term @ final_state).item()}")
print(f"<H> = {(final_state.T @ model.hamiltonian @ final_state).item()}")

# QLM Lattice 6x4

In [ ]:
coup_j, coup_rk = (1, -0.7)  # dfs 3 mins 33 secs
basis, model = setup_link_model(lattice_shape=(6, 4), n_solution=32810, coup_j=coup_j, coup_rk=coup_rk) 

In [ ]:
# basis.dataframe.to_parquet("qlm_6x4_lattice.parquet", index=False)
basis = ComputationBasis.from_parquet("data/qlm_6x4_lattice.parquet")
coup_j, coup_rk = (1, -0.7)
model = QuantumLinkModel(coup_j, coup_rk, (6, 4), basis)

In [ ]:
evals, evecs = np.load("data/qlm_6x4_coup_j_1_coup_rk_-0.7_eigs.npz").values()

In [ ]:
evecs_df = pd.read_parquet("data/qlm_6x4_coup_j_1_coup_rk_-0.7_eigs.parquet")
evecs_df[evecs_df["kin"].abs() < 1e-12]

In [ ]:
evecs_df[(evecs_df["kin"].abs() < 1e-12) & ((evecs_df["pot"].abs() - 10).abs() < 1e-12)]

In [ ]:
i = 17481
plt.plot(evecs[:, i], linestyle='--', marker='.')
# np.where(np.abs(evecs[:, i]) > 0.04)[0]

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    # np.where(np.diag(two_steps_mat) == 12)[0],
    # np.where(np.diag(two_steps_mat) == 10)[0],
    # np.where(np.abs(evecs[:, 17479]) > 0.03)[0]
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    # "yellowgreen",
    # "tomato",
    # "royalblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 12)[0])
igraph.plot(
    sub_ig,
    layout=sub_ig.layout_kamada_kawai(),
    vertex_size=4,
    # vertex_color="orange",
    edge_width=0.2,
    edge_color="darkgray",
    vertex_label_size=1,
    inline=True
)

In [ ]:
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 10)[0])
igraph.plot(
    sub_ig,
    layout=sub_ig.layout_kamada_kawai(),
    vertex_size=4,
    # vertex_color="orange",
    edge_width=0.2,
    edge_color="darkgray",
    vertex_label_size=1,
    inline=True
)

In [ ]:
two_steps_mat = csr_matrix(model.kinetic_term) ** 2
# sort_idx = np.argsort(np.diag(two_steps_mat))

# plt.plot(np.diag(two_steps_mat[sort_idx, :][:, sort_idx]), linestyle="--", marker="o")
# plt.show()
np.unique(two_steps_mat.diagonal())
two_steps_mat = two_steps_mat.toarray()

In [ ]:
g = nx.from_numpy_array(two_steps_mat)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    np.where(np.abs(evecs[:, 17475]) > 0.03)[0]
]
highlight_color = [
    # "aqua",
    # "deepskyblue",
    # "yellowgreen",
    "tomato",
    # "royalblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 10)[0])
igraph.plot(
    sub_ig,
    # layout=sub_ig.layout_kamada_kawai(),
    vertex_size=4,
    # vertex_color="orange",
    edge_width=0.2,
    edge_color="darkgray",
    vertex_label_size=1,
    inline=True
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

for c in sub_components:
    mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
    print(mat.shape[0] - np.linalg.matrix_rank(mat))
    
print([len(c) for c in sub_components])

In [ ]:
g = nx.from_numpy_array(-model.kinetic_term)
ig = igraph.Graph.from_networkx(g)
ig.vs["label"] = [str(i) for i in range(ig.vcount())]
color = ["whitesmoke" for i in range(ig.vcount())]
highlight = [
    list(map(int, sub_ig.subgraph(sub_components[0]).vs["label"])),
    list(map(int, sub_ig.subgraph(sub_components[1]).vs["label"])),
    list(map(int, sub_ig.subgraph(sub_components[2]).vs["label"])),
    # np.where(np.diag(two_steps_mat) == 10)[0]
]
highlight_color = [
    "tomato",
    "yellowgreen",
    "royalblue"
]
for i, nodes in enumerate(highlight):
    for node in nodes:
        color[node] = highlight_color[i]
ig.vs["color"] = color
sub_ig = ig.induced_subgraph(np.where(np.diag(two_steps_mat) == 10)[0])
igraph.plot(
    sub_ig,
    # layout=sub_ig.layout_kamada_kawai(),
    vertex_size=4,
    # vertex_color="orange",
    edge_width=0.2,
    # edge_color="gray"
    vertex_label_size=1,
)

In [ ]:
sub_components = sub_ig.connected_components(mode="weak")

nullity = []
for c in sub_components:
    if len(c) > 1:
        mat = nx.to_numpy_array(sub_ig.subgraph(c).to_networkx())
        nullity.append(mat.shape[0] - np.linalg.matrix_rank(mat))

print(np.unique(nullity, return_counts=True))
print(np.unique([len(c) for c in sub_components], return_counts=True))